In [ ]:
import requests

# Generell hjelpefunksjon for å hente metadata fra Concept LDS Rest API på formen "https://concept-lds.staging-bip-app.ssb.no/ns/<domen-objekt>/<id>"
def get_lds_metadata(domain_and_id):
    lds_result = requests.get('https://concept-lds.staging-bip-app.ssb.no/ns' + domain_and_id)
    return lds_result.json()

## Eksempel på hvordan funksjonen kan brukes til å hente ut domene-objekter med ID i LDS.
#print(get_lds_metadata('/RepresentedVariable/f2bdc3f3-5275-40a6-897d-c00a494532fc'))
#print(get_lds_metadata('/Variable/3335298a-779d-4bd4-84a1-3262b8afbbeb'))
#print(get_lds_metadata('/EnumeratedValueDomain/5b30542b-3ed0-4136-8d1e-ebb5097d35af'))
#print(get_lds_metadata('/Population/96869bb9-ebcd-4630-ab73-005a4c4b4674'))
#print(get_lds_metadata('/Universe/6ab622e1-adf1-4739-b549-c482bd901c4d'))
    

In [16]:
import json

dataset_name = ""
dataset_description = ""
dataset_unit_type = ""
identifier_instance_variables = []
time_instance_variables = []
measure_instance_variables = []
attribute_instance_variables = []

# Leser skatte-metadata-json-filen og transformer den til Python dict (dictionary objekt)
with open('skattedata_metadata.json') as meta_json:
    metadata = json.load(meta_json)
    
    dataset_name = metadata['name']
    dataset_description = metadata['description']

    selectedUnitType = metadata['unitType']['selected-id']
    for unitType in metadata['unitType']['candidates']:
        if unitType['id'] == selectedUnitType:
            dataset_unit_type = unitType['name'].upper()            
                
    for instVar in metadata['instanceVariables']:
        #print(" " + instVar['name'])
        #print("   " + instVar['dataStructureComponentType']['selected-enum'])
        if instVar['dataStructureComponentType']['selected-enum'] == 'IDENTIFIER':
            identifier_instance_variables.append(instVar)
        elif instVar['dataStructureComponentType']['selected-enum'] == 'MEASURE' and instVar['dataStructureComponentRole']['selected-enum'] == 'TIME':
            time_instance_variables.append(instVar)
        elif instVar['dataStructureComponentType']['selected-enum'] == 'MEASURE':
            measure_instance_variables.append(instVar)
        else:
            attribute_instance_variables.append(instVar)


In [34]:
from datetime import date

# Henter relevant informasjon om identifier-variabel
# OBS! Forutsetter her at det kun er 1 indentifier i skattedatasettet!
indentifier_name = identifier_instance_variables[0]['name'].upper()
indentifier_representedVar_id = identifier_instance_variables[0]['representedVariable']['selected-id']

# For hver "Measure-InstanceVariable" i skattedatasettet bygges det opp et Microdata-metadata-dokument (json-dokument)
for measure_instVar in measure_instance_variables:
    population_id = measure_instVar['population']['selected-id']
    # Finner Population i Concetp LDS med denne ID
    lds_population = get_lds_metadata('/Population/' + population_id)
    population_description = lds_population['description'][0]['languageText']
    #TODO: Vi mangler temporalityType i JSON-filen og LDS !!!!!
    temporality_type = "ACCUMULATED"
    
   
    # Bygger metadata til Microdata.no (JSON-format som brukes til import av metadata i Microdata).
    microdata_dataset_json = f'''\
{{
    "language": "nb",
    "name": "{dataset_name}",
    "title": "Renteinntekter",
    "description": "Renteinntekter av innenlandske bankinnskudd samt utestående fordringer i løpet av kalenderåret.\n Renteinntekter omfatter renter av bankinnskudd i norske banker. Beløpet omfatter også renteinntekter på norske ihendehaverobligasjoner, panteobligasjoner og andre gjeldsbrev.\n Variabelen omfatter bosatte per 31.12.ÅÅÅÅ. Personer med verdiene 0 og uoppgitt (missing) er utelatt.",
    "identifierName": [
        "{indentifier_name}"
    ],
    "attributeName": [
        "START",
        "STOP"
    ],
    "populationDescription": "{population_description}",
    "temporalityType": "{temporality_type}",
    "spatialCoverageDescription": "Norge",
    "measure": {{
        "name": "INNTEKT_RENTINNT",
        "title": "Renteinntekter",
        "description": "Renteinntekter av innenlandske bankinnskudd samt utestående fordringer i løpet av kalenderåret.\n Renteinntekter omfatter renter av bankinnskudd i norske banker. Beløpet omfatter også renteinntekter på norske ihendehaverobligasjoner, panteobligasjoner og andre gjeldsbrev.\n Variabelen omfatter bosatte per 31.12.ÅÅÅÅ. Personer med verdiene 0 og uoppgitt (missing) er utelatt.",
        "subjectFieldName": [
            "INNTEKT"
        ],
        "unitTypeName": null,
        "dataType": "LONG",
        "uriDefinition": [
            "https://www.ssb.no/a/metadata/conceptvariable/vardok/560/nb"
        ],
        "valueDomainName": "BELOEP_NOK"
    }},
    "dataRevision": {{
        "releaseDate": "2017-01-01",
        "releaseStatus": "OK",
        "description": "Første publisering",
        "temporalCoverageStart": "1993-01-01",
        "temporalCoverageLatest": "2015-12-31",
        "temporalEndOfSeries": false
    }},
    "ssbProperty": [
        {{
            "key": "KILDE_FIL",
            "value": "DAPLA - {dataset_name}"
        }},
        {{
            "key": "GENERERT",
            "value": "{str(date.today())}"
        }}
    ]
}}
'''

    print(microdata_dataset_json)
    print()
    print()
    print()



{
    "language": "nb",
    "name": "skattedata",
    "title": "Renteinntekter",
    "description": "Renteinntekter av innenlandske bankinnskudd samt utestående fordringer i løpet av kalenderåret.
 Renteinntekter omfatter renter av bankinnskudd i norske banker. Beløpet omfatter også renteinntekter på norske ihendehaverobligasjoner, panteobligasjoner og andre gjeldsbrev.
 Variabelen omfatter bosatte per 31.12.ÅÅÅÅ. Personer med verdiene 0 og uoppgitt (missing) er utelatt.",
    "identifierName": [
        "PSEUDOIDENT"
    ],
    "attributeName": [
        "START",
        "STOP"
    ],
    "populationDescription": "Alle personer med skatteplikt til Norge for kalenderåret 2019",
    "temporalityType": "ACCUMULATED",
    "spatialCoverageDescription": "Norge",
    "measure": {
        "name": "INNTEKT_RENTINNT",
        "title": "Renteinntekter",
        "description": "Renteinntekter av innenlandske bankinnskudd samt utestående fordringer i løpet av kalenderåret.
 Renteinntekter omfatter

In [ ]:
import requests

# Henter concept-metadata via Concept LDS sitt API
#result = requests.get('https://concept-lds.staging-bip-app.ssb.no/ns/RepresentedVariable/' + 'f2bdc3f3-5275-40a6-897d-c00a494532fc')
#print(result.text)

# Finner rep
for instVar in instance_variables:
    represented_variable_id = instVar['representedVariable']['selected-id']
    lds_result = requests.get('https://concept-lds.staging-bip-app.ssb.no/ns/RepresentedVariable/' + represented_variable_id)
    lds_represented_variable = lds_result.json()
    print(lds_represented_variable['shortName'])
    print(lds_represented_variable['name'][0]['languageText'])
    print(lds_represented_variable['description'][0]['languageText'])
    print(lds_represented_variable['substantiveValueDomain'])
    print(lds_represented_variable['universe'])
    print()